In [7]:
import tkinter as tk
from tkinter import ttk
from reportlab.pdfgen import canvas

def print_invoice():
    # Retrieve selected item from TreeView
    selected_item = tree.selection()

    if not selected_item:
        print("No item selected")
        return

    # Get values of selected item
    values = tree.item(selected_item)['values']

    # Extract data
    date = values[0]
    invoice_number = values[1]
    item_name = values[2]
    quantity = values[3]
    price = values[4]
    total_amount = values[5]

    # Create PDF invoice
    pdf_filename = f"Invoice_{invoice_number}.pdf"
    c = canvas.Canvas(pdf_filename)

    # Set up PDF content
    c.drawString(100, 750, "Invoice")
    c.drawString(100, 730, f"Date: {date}")
    c.drawString(100, 710, f"Invoice Number: {invoice_number}")
    c.drawString(100, 690, f"Item Name: {item_name}")
    c.drawString(100, 670, f"Quantity: {quantity}")
    c.drawString(100, 650, f"Price: {price}")
    c.drawString(100, 630, f"Total Amount: {total_amount}")

    # Save PDF file
    c.save()

    print(f"PDF invoice created: {pdf_filename}")

# Create Tkinter window
root = tk.Tk()
root.title("PDF Invoice Generator")

# Create TreeView
columns = ("Date", "Invoice Number", "Item Name", "Quantity", "Price", "Total Amount")
tree = ttk.Treeview(root, columns=columns, show="headings")

# Insert sample data
sample_data = [("2024-02-01", "INV001", "Product A", 2, 25.00, 50.00),
               ("2024-02-02", "INV002", "Product B", 1, 30.00, 30.00),
               ("2024-02-03", "INV003", "Product C", 3, 15.00, 45.00)]

for data in sample_data:
    tree.insert("", "end", values=data)

# Add TreeView column headings
for col in columns:
    tree.heading(col, text=col)
    tree.column(col, anchor="center", width=100)

# Create Print PDF Invoice button
print_button = tk.Button(root, text="Print PDF Invoice", command=print_invoice)
print_button.pack(pady=10)

# Pack the TreeView
tree.pack()

# Run the Tkinter event loop
root.mainloop()


FileNotFoundError: [Errno 2] No such file or directory: 'main_ttk\\sheets-to-python-credential.json'